In [ ]:
import pandas as pd
import numpy as np

dataset_20_21 = '../web_scraping/matches.csv'
data = pd.read_csv(dataset_20_21)
data_df = data[["date", "venue", "team", "opponent", "gf", "ga", "result", "xg", "sot","poss"]]
data_df["date"] = pd.to_datetime(data_df["date"], format="%Y-%m-%d")
data_df["opponent"] = data_df["opponent"].str.replace("Utd", "United").replace("West Ham", "West Ham United").replace("Nott'ham Forest", "Nottingham Forest").replace("Tottenham", "Tottenham Hotspur").replace("Wolves", "Wolverhampton Wanderers").replace("Brighton", "Brighton and Hove Albion").replace("West Brom", "West Bromwich Albion");
data_df["Score"] = data_df["gf"] -  data_df["ga"]
data_df["Round"] = np.zeros((len(data_df), 1))

current_round = 1
for row in range(1,len(data)+1):
  data_df['Round'][row-1] = current_round
  if (row % 10) == 0:
    current_round += 1

teamsList= {k:v for v, k in enumerate(np.sort(data_df["team"].unique()))}
data_df["teamId"] = [teamsList[id] for id in list(data_df["team"].values)]
data_df["opponentId"] = [teamsList[id] for id in list(data_df["opponent"].values)]
data_df["duplicate"] = False

data_df = data_df.sort_values(by=["date"], ascending=False)
data_df = data_df.loc[(data_df["teamId"] == 12) | (data_df["opponentId"] == 12)]



for i in range(len(data_df)):
  fixture = data_df.iloc[i]
  if(fixture["duplicate"] == False):
    duplicate = data_df.index[(data_df["date"] == fixture["date"]) & (data_df["teamId"] == fixture["opponentId"]) & (data_df["opponentId"] == fixture["teamId"])]
    if(data_df.at[duplicate[0], "duplicate"] == False):
        data_df.at[duplicate[0], "duplicate"] = True

data_df= data_df.loc[data_df["duplicate"] == False]

data_df



In [ ]:
import torch
def get_last_5_matches(row):
    last_matches = data_df.loc[data_df["date"] < row["date"]].head(5);    
    array = last_matches[["gf","ga","xg", "sot", "poss"]].to_numpy();
    if(len(last_matches) <= 5):
        base = np.zeros((5,5))
        for i in range(len(last_matches)):
            base[i,:] = last_matches.iloc[i][["gf","ga","xg", "sot", "poss"]].to_numpy()
        array = base
        tensor = torch.from_numpy(array) 
    row["last_5_matches"] = tensor
    return row   
    
data_df= data_df.apply(get_last_5_matches, axis=1)

data_df[["date","team","opponent", "last_5_matches"]]